## Basic prompty sample
- Chat with prompty
- Add weather function
- http request using natural language
- code interpreter (powershell)

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.22.0"
#r "nuget: Microsoft.SemanticKernel.Prompty, 1.22.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.22.0"

#r "nuget:AutoGen,0.2.1"
#r "nuget:AutoGen.DotnetInteractive,0.2.1"

Installed Packages AutoGen, 0.2.1 AutoGen.DotnetInteractive, 0.2.1 Microsoft.SemanticKernel, 1.22.0 Microsoft.SemanticKernel.Abstractions, 1.22.0 Microsoft.SemanticKernel.Prompty, 1.22.0-alpha

In [2]:
Start-Process pwsh -ArgumentList "-c dotnet run --project .\Interactive.Prompty\WeatherService\WeatherForecastingService.csproj"

In [4]:
#r "Interactive.Prompty/Interactive.Prompty/bin/Debug/net8.0/Interactive.Prompty.dll"

In [5]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.ComponentModel;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.Commands;
using SKKernel = Microsoft.SemanticKernel.Kernel;
using Kernel = Microsoft.DotNet.Interactive.Kernel;
using Interactive.Prompty;
using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using AutoGen.DotnetInteractive;
using AutoGen.DotnetInteractive.Extension;


await PromptyExtension.LoadExtensionAsync((CompositeKernel)Microsoft.DotNet.Interactive.Kernel.Root);

In [5]:
#!connect prompty --kernel-name prompty

Kernel added: #!prompty

In [7]:
---
name: ExamplePrompt
description: A prompt that uses context to ground an incoming question
authors:
  - HeHe
model:
  api: chat
sample:
  firstName: Geno
  context: empty
---
 
system:
You are an AI assistant who helps people with_anything. As the assistant,
you answer questions briefly, succinctly, and in a personable manner using markdown and even add some personal flair with appropriate emojis. You try to resolve customer's question in one-step

The following http services are available. You can make request to these services using http request syntax

## weather forecast service
- base address: http://localhost:5049
- swagger
```json
{
    "openapi": "3.0.1",
    "info": {
        "title": "WeatherForecastingService",
        "version": "1.0"
    },
    "paths": {
        "/weatherforecast": {
            "get": {
                "tags": [
                    "WeatherForecastingService"
                ],
                "operationId": "GetWeatherForecast",
                "responses": {
                    "200": {
                        "description": "OK",
                        "content": {
                            "application/json": {
                                "schema": {
                                    "type": "array",
                                    "items": {
                                        "$ref": "#/components/schemas/WeatherForecast"
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    },
    "components": {
        "schemas": {
            "WeatherForecast": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "format": "date"
                    },
                    "temperatureC": {
                        "type": "integer",
                        "format": "int32"
                    },
                    "summary": {
                        "type": "string",
                        "nullable": true
                    },
                    "temperatureF": {
                        "type": "integer",
                        "format": "int32",
                        "readOnly": true
                    }
                },
                "additionalProperties": false
            }
        }
    }
}
```
 
# Context
Use the following context to provide a more personalized response to {{firstName}}:
{{context}}
 
user:
{{input}}

In [6]:
#!connect prompty-orchestrator --kernel-name orchestrator-kernel --prompty-kernel-name prompty --azure-openai-endpoint @input:{"saveAs":"azure-openai-endpoint"} --azure-openai-deployment-name @input:{"saveAs":"azure-openai-deployment-name"} --azure-openai-apikey @password:{"saveAs":"azure-openai-apikey"}

Please enter a value for parameter: --azure-openai-endpoint Please enter a value for parameter: --azure-openai-deployment-name Please enter a value for parameter: --azure-openai-apikey Ok

Kernel added: #!orchestrator-kernel

In [8]:
Who are you helping with

I'm here to assist you with anything you need! 😊 Just let me know what you need help with.

In [32]:
[KernelFunction]
[Description("Run Powershell")]
public async static Task<string> Powershell([Description("the powershell script to run")] string pwsh)
{
    var pwshKernel = Kernel.Root.FindKernelByName("pwsh");
    $"generated code {pwsh}".Display();
    var command = new SubmitCode(pwsh);
    List<KernelEvent> events = new List<KernelEvent>();
    var sub = Kernel.Root.KernelEvents.Subscribe(e => events.Add(e));
    var result = await pwshKernel.SendAsync(command);   
    sub.Dispose();
    var displayValues = events
                        .OfType<DisplayEvent>()
                        .SelectMany(x => x.FormattedValues);

    var response =  string.Join("\n", displayValues.Select(x => x.Value)) ?? "Nothing to display";
    $"results : {response}".Display();
    return response;
}

[KernelFunction]
[Description("Run http request")]
public async static Task<string> Http([Description("the HTTP request to run")] string http)
{
    var httpKernel = Kernel.Root.FindKernelByName("http"); 
    $"generated code {http}".Display();
    var command = new SubmitCode(http);
    List<KernelEvent> events = new List<KernelEvent>();
    var sub = Kernel.Root.KernelEvents.Subscribe(e => events.Add(e));
    var result = await httpKernel.SendAsync(command);
    sub.Dispose();
    var displayValues = events
                        .OfType<DisplayEvent>()
                        .SelectMany(x => x.FormattedValues);

    var response = string.Join("\n", displayValues.Select(x => x.Value)) ?? "Nothing to display";
    $"results : {response}".Display();
    return response;
}

In [33]:
show me the how many files are in current folder using powershell, list their names

You've got 7 files in the current folder. Here are their names:

1. `.gitignore`
2. `basic.prompty`
3. `NuGet.config`
4. `README.md`
5. `sample.ipynb`
6. `z3.ipynb`
7. `z3.prompty`

📂✨

generated code Get-ChildItem -File | Select-Object Name, @{Name='Count';Expression={($_ | Measure-Object).Count}}


Name          Count
----          -----
.gitignore        1
basic.prompty     1
NuGet.config      1
README.md         1
sample.ipynb      1
z3.ipynb          1
z3.prompty        1



results : 

Name          Count

----          -----

.gitignore        1

basic.prompty     1

NuGet.config      1

README.md         1

sample.ipynb      1

z3.ipynb          1

z3.prompty        1




In [34]:
use http to create a weather forecasting report for me

Here's your weather forecast for the next five days! 🌤️

| Date       | Temperature (°C) | Temperature (°F) | Summary |
|------------|-------------------|------------------|---------|
| 2024-10-18 | 44                | 111              | Mild    |
| 2024-10-19 | 15                | 58               | Balmy   |
| 2024-10-20 | 10                | 49               | Mild    |
| 2024-10-21 | 40                | 103              | Mild    |
| 2024-10-22 | 54                | 129              | Mild    |

Enjoy your days ahead! ☀️

generated code GET http://localhost:5049/weatherforecast

Name,Value
traceparent,00-80f8f7ac3798c8402c36b1b76d8230ca-85b0fec178651dc4-00
Name,Value
Date,"Thu, 17 Oct 2024 21:19:42 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,380
date,"""2024-10-18"""
temperatureC,44
summary,"""Mild"""


results : <div class="http-response-message-container"><style type="text/css">
            .http-response-message-container {
                display: flex;
                flex-wrap: wrap;
            }

            .http-response-message-container > div {
                margin: .5em;
                padding: 1em;
                border: 1px solid;
            }

            .http-response-message-container > div > h2 {
                margin-top: 0;
            }

            .http-response-message-container > div > h3 {
                margin-bottom: 0;
            }

            .http-response-message-container summary {
                margin: 1em 0;
                font-size: 1.17em;
                font-weight: 700;
            }

            @keyframes blink {
                0% {
                  opacity: .2;
                }
                20% {
                  opacity: 1;
                }
                100% {
                  opacity: .2;
                }
        